In [18]:
import numpy as np 
import pandas as pd  
import math 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from tqdm import tqdm

In [19]:
tokenizer = AutoTokenizer.from_pretrained("totoro4007/cryptodeberta-base-all-finetuned") 
model = AutoModelForSequenceClassification.from_pretrained("totoro4007/cryptodeberta-base-all-finetuned") 
model.cuda() 
model.eval() 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DebertaForSequenceClassification(
  (deberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(52001, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermed

In [20]:
news_df = pd.read_csv("august_2022_coinness.csv") 

In [ ]:
device = torch.device("cuda") 

news_sentiment_dict = {} 

titles = news_df["titles"].values 
contents = news_df["contents"].values 
years = news_df["year"].values 
months = news_df["month"].values 
days = news_df["day"].values 
hours = news_df["hour"].values 

m = nn.Softmax(dim=1) 

for i in range(len(years)):
    datestr = str(years[i]) + '/' + str(months[i]) + '/' + str(days[i]) + '/' + str(hours[i]) 
    news_sentiment_dict[datestr] = [] 

for i in tqdm(range(len(years)), desc="calculating news sentiment scores", position=0, leave=True):
    datestr = str(years[i]) + '/' + str(months[i]) + '/' + str(days[i]) + '/' + str(hours[i]) 
    title = titles[i] 
    content = contents[i] 
    inputs = tokenizer(title, content, return_tensors="pt", max_length=512, padding="max_length", truncation=True).to(device)
    
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = m(logits)
    probs = probs.detach().cpu().numpy().flatten() 
    
    news_sentiment_dict[datestr].append(probs) 


calculating news sentiment scores:   0%|          | 36/101919 [00:02<1:54:38, 14.81it/s]

In [ ]:
import pickle

with open('news_sentiment_dict.pickle', 'wb') as handle:
    pickle.dump(news_sentiment_dict, handle)

with open('news_sentiment_dict.pickle', 'rb') as handle:
    b = pickle.load(handle)



In [ ]:
b